In [ ]:
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
import os
from tqdm import tqdm_notebook as tqdm
import time
from lxml import etree
from sklearn.metrics import r2_score
from datetime import timedelta
import numpy as np

In [ ]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def pretty_print_result(search_result, fields=None):
    if fields is None:
        fields = []
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
        for field in fields:
            print(f'{field}: {hit["_source"][field]}')


def get_score(search_result):
    res = []
    for hit in search_result['hits']['hits']:
        res.append((hit["_id"], hit["_score"]))
    res.sort(key = lambda x: x[1], reverse = True)
    return res


class Index:
    def __init__(self, index, settings):
        self.index_name = index
        self.settings = settings
        self.es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360}])
        if self.es.indices.exists(index=index):
            self.es.indices.delete(index=index)
        self.es.indices.create(index=index, body=settings)

    def es_actions_generator(self, path_to_docs):
        for doc_name in tqdm(os.listdir(path_to_docs)):
            with open(f"{path_to_docs}/{doc_name}", "r", encoding="utf-8") as inf:
                doc_id = int(''.join(list(filter(str.isdigit, doc_name))))
                doc = json.load(inf)           
            yield create_es_action(self.index_name, doc_id, doc)


    def add_documents(self, path_to_docs):
        try:
            for ok, result in parallel_bulk(self.es, self.es_actions_generator(path_to_docs), queue_size=4, thread_count=4,
                                        chunk_size=1000):
                  if not ok:
                     print(result)
        except Exception as e: 
            print(e)

    def get_doc_by_id(self, doc_id):
        return self.es.get(index=self.index_name, id=doc_id)['_source']

    def search(self, query, *args):
        return self.es.search(index=self.index_name, body=query, size=20)
        # note that size set to 20 just because default value is 10 and we know that we have 12 docs and 10 < 12 < 20



In [ ]:
def print_index_size(index): 
    print(f"{(index.es.indices.stats(index.index_name)['_all']['primaries']['store']['size_in_bytes'] / 2 ** 30):.2f} GB")

In [ ]:
settings_1 = {
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            }
        }
    }
}

In [ ]:
settings = {
        'mappings': {
            'properties': {
                'text': {
                    'type': 'text',
                    'analyzer': 'russian_complex',
                    'search_analyzer': 'russian_complex'
                }
            }
            
        },
        "settings": {
        "analysis" : {
            "analyzer" : {
                "my_analyzer" : {
                    "tokenizer" : "standard",
                    "filter" : ["lowercase", "russian_snow", "english_snow"]
                },
                'russian_complex': {
                    'char_filter': [
                        'yont'
                    ],
                    'tokenizer': 'word_longer_2',
                    'filter': [
                        'lowercase',
                        'russian_snow'
                    ]
                }
            },
            'char_filter': {
                'yont': {
                    'type': 'mapping',
                    'mappings': [
                        'ё => е'
                    ]
                }
            },
            'tokenizer': {
                'word_longer_2': {
                    'type': 'pattern',
                    'pattern': '[a-zA-Z_0-9\u0400-\u04FF]{2,}',
                    'group': 0
                },
                'white_20': {
                    'type': 'whitespace',
                    'max_token_length': 5
                }
            },
            "filter" : {
                "russian_snow" : {
                    "type" : "snowball",
                    "language" : "Russian"
                },
                "english_snow" : {
                    "type" : "snowball",
                    "language" : "English"
                }
            }
        }
    }
}

In [ ]:
index = Index("docs", settings_1)

In [ ]:
start = time.time()
index.add_documents("data/json_text")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))

In [ ]:
print_index_size(index)

In [ ]:
stem_index = Index('stem_docs', settings)

In [ ]:
start = time.time()
stem_index.add_documents("res/stemmed_titles")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))

In [ ]:
print_index_size(stem_index)

In [ ]:
stem_index.get_doc_by_id('1000039')

In [ ]:
class Query:
    def __init__(self, task_id, query, relevant_docs):
        self.task_id = task_id
        self.query = query
        self.relevant_docs = relevant_docs


In [ ]:

def json_query(query):
    return {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'text': query.query
                    }
                }
            ]
        }
    }
}
        


In [ ]:
def pagerank_query(query):
    return  {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'content': query.query
                    }
                },
                {
                    'match': {
                        'title': {
                            'query': query.query,
                            'boost': '0.15'
                        }
                    }
                },
                {
                    'rank_feature': {
                        'field': 'pagerank',
                        'saturation': {
                            'pivot': 10
                        },
                        'boost': '0.15'
                    }
                }
            ]
        }
    }
}

In [ ]:
def all_query(query):
    return  {
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'content': query.query
                    }
                },
                {
                    'match': {
                        'title': {
                            'query': query.query,
                            'boost': '0.15'
                        }
                    }
                },
                {
                    'rank_feature': {
                        'field': 'pagerank',
                        'saturation': {
                            'pivot': 10
                        },
                        'boost': '5'
                    }
                },
                {
                    'rank_feature': {
                        'field': 'urllen',
                        'boost': '0.5'
                    }
                },
                {
                    'rank_feature': {
                        'field': 'doclen',
                        'boost': '0.15'
                    }
                }
            ]
        }
    }
}

In [ ]:
class Metrics:
    def __init__(self, p, r, r_precision, map_score):
        self.r = r
        self.p = p
        self.r_precision = r_precision
        self.map_score = map_score
    
    def __str__(self):
        return f"r = {self.r}\np = {self.p}\nr_precision = {self.r_precision}\nMAP = {self.map_score}"
    
    __repr__ = __str__


class SearchQualityChecker:
    def __init__(self, queries, index):
        self.queries = queries
        self.index = index
        self.results = {}
        self.metrics = {}
        
    def get_results(self, get_query=json_query):
        r_precision_total = 0
        map_score_total = 0
        r_total = 0
        p_total = 0
        for q in tqdm(self.queries):
            res = self.index.search(get_query(q))
            print(q.task_id)
            pretty_print_result(res)
            scores = get_score(res)
            metric = Metrics(p=self.p(20, q, scores), r=self.r(20, q, scores), r_precision=self.r_precision(q, scores),
                            map_score=self.map_score(q, scores, 20))
            p_total += metric.p
            r_total += metric.r
            r_precision_total += metric.r_precision
            map_score_total += metric.map_score
            self.metrics[q.task_id] = metric
        Q = len(self.queries)
        print(Q)
        return Metrics(p=p_total / Q, r=r_total / Q, r_precision=r_precision_total / Q, map_score=map_score_total / Q)
    
    def r_precision(self, query, search_res_score):
        return self.r(len(query.relevant_docs), query, search_res_score)
    
    def map_score(self, query, search_res_score, n):
        m = 0
        for k in range(1, n):       
            m += self.p(k, query, search_res_score)
        R = len(query.relevant_docs)
        return m / n
    
    def p(self, k, query, search_res_score):
        r = 0
        for doc, _ in search_res_score[:k]:
            if doc in query.relevant_docs:
                r += 1
        return r / k
    
    def r(self, k, query, search_res_score):
        R = len(query.relevant_docs)
        r = 0
        for doc, _ in search_res_score[:k]:
            if doc in query.relevant_docs:
                r += 1
        return r / R if R != 0 else 0 if len(search_res_score) > 0 else 1



In [ ]:
def find_diff_metrics(quality_checker, other_checker, k=20, comp=lambda x : x.map_score):
     res = []
     for task_id in quality_checker.metrics:
        metric = quality_checker.metrics[task_id]
        other_metric = other_checker.metrics[task_id]
        res.append((task_id, abs(comp(metric) - comp(other_metric)), metric, other_metric))
     res.sort(reverse=True, key=lambda x: x[1])
     return res

In [ ]:
from collections import defaultdict
def get_relevance():
    res = defaultdict(list)
    xml_tree = etree.parse("data/relevant_table_2009.xml")
    root = xml_tree.getroot()
    for task in root.getchildren():
        relevant_docs = set()
        for document in task.getchildren():
            if document.get("relevance") == "vital":
                relevant_docs.add(document.get("id"))
        res[task.get("id")] = relevant_docs
    print(len(res))
    return res


def generate_queries_plain_texts():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in root.getchildren():
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    if len(relevances[task.get("id")]) > 0:
                        res.append(Query(task.get("id"), query_text.text, relevances[task.get("id")]))
                except Exception as e:
                    print(e)
    print(len(res))
    return res

In [ ]:
queries = generate_queries_plain_texts()
#print(queries)
quality_checker = SearchQualityChecker(queries, index)
plain_text_res = quality_checker.get_results()

In [ ]:
plain_text_res

In [ ]:
from pymystem3 import Mystem
from nltk.corpus import stopwords
import nltk
import re
#nltk.download('punkt')
#nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import RussianStemmer

russian_stopwords = stopwords.words("russian")
english_stopwords = stopwords.words("english")
black_list = ["°", "№", "©", "...", "//", "://", "</", "\">", "=\"", "=\'", "\r", "\n", "\t"]
stem = Mystem()

def lemmatize(text):
    words = nltk.word_tokenize(text.lower())
    tokens = []
    for word in words:
        tokens.extend(stem.lemmatize(word))
    tokens = [token for token in tokens if token != " " and token.strip() not in punctuation \
              and token not in russian_stopwords and token not in english_stopwords \
              and token not in black_list \
              and token.find("\r") == -1 \
              and token.find("\n") == -1 \
              and token.find("\t") == -1 \
              and not (token.isdigit() and len(token) == 1)]
    return ' '.join(tokens)

def stemmize(text):
    english_check = re.compile(r'[a-z]')
    stemmerr = RussianStemmer()
    stemmere = PorterStemmer()
    
    lowercase_text = text.lower()

    tokens = []
    for token in word_tokenize(lowercase_text):
        if english_check.match(token):
            tokens.append(stemmere.stem(token))
        else:
            tokens.append(stemmerr.stem(token))

    tokens = [token for token in tokens if token != " " and token.strip() not in punctuation \
              and token not in russian_stopwords and token not in english_stopwords]
    return ' '.join(tokens)

def generate_queries_lemmas():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in tqdm(root.getchildren()):
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    if len(relevances[task.get("id")]) > 0:
                        res.append(Query(task.get("id"), lemmatize(query_text.text), relevances[task.get("id")]))
                except:
                    pass
    print(len(res))
    return res

def generate_queries_stem():
    relevances = get_relevance()
    xml_tree = etree.parse("data/web2008_adhoc.xml")
    root = xml_tree.getroot()
    res = []
    for task in tqdm(root.getchildren()):
        if task.get("id") is not None:
            for query_text in task.getchildren():
                try:
                    res.append(Query(task.get("id"), stemmize(query_text.text), relevances[task.get("id")]))
                except:
                    pass
    print(len(res))
    return res

In [ ]:
queries_lemmas = generate_queries_lemmas()

In [ ]:
queries_lemmas[8].query

In [ ]:
lemma_index = Index("lemma_docs", settings_1)

start = time.time()
lemma_index.add_documents("data/json_filtered_tokens_texts")
elapsed = time.time() - start
print(str(timedelta(seconds=elapsed)))

In [ ]:
lemma_quality_checker = SearchQualityChecker(queries_lemmas, lemma_index)
lemma_res = lemma_quality_checker.get_results()

In [ ]:
lemma_res

In [ ]:
print_index_size(lemma_index)

In [ ]:
find_diff_metrics(quality_checker, lemma_quality_checker)

In [ ]:
queries_stem = generate_queries_stem()

In [ ]:
id_to_pagerank = {}
with open('data/pagerank.txt','r') as f:
    for line in f:
        docId, docURL, rank = line.split()
        id_to_pagerank[int(docId)] = float(rank)

In [ ]:
len(id_to_pagerank)


In [ ]:
 for doc_name in tqdm(os.listdir("data/json_filtered_tokens_texts")):
        with open(f"data/json_filtered_tokens_texts/{doc_name}", "r+", encoding="utf-8") as inf:
            doc_id = int(''.join(list(filter(str.isdigit, doc_name))))
            doc = json.load(inf)
            try:
                doc["pagerank"] = id_to_pagerank.get(doc_id)
            except:
                pass
            inf.seek(0)        # <--- should reset file position to the beginning.
            json.dump(doc, inf, indent=4, ensure_ascii=False)
            inf.truncate()

In [ ]:
settings_with_pagerank = {
    "mappings": {
        "properties": {
            "title": {
                "type": "text"
            },
            "content": {
                "type": "text"
            },
            "pagerank": {
                "type": "rank_feature"
            }
        }
    }
}

In [ ]:
pr_index = Index("pagerank_index", settings_with_pagerank)

In [ ]:
pr_index.add_documents("res/lemmatized_titles_pr")

In [ ]:
print_index_size(pr_index)

In [ ]:
pr_quality_checker = SearchQualityChecker(queries_lemmas, pr_index)
pr_res = pr_quality_checker.get_results(pagerank_query)

In [ ]:
pr_res

In [ ]:
pr_index.get_doc_by_id(1000039)

In [ ]:
settings_titles = {
        'mappings': {
            'properties': {
                'content': {
                    'type': 'text',
                },
                'title': {
                    'type': 'text'
                }
            }
            
        }
    }

In [ ]:
pr_quality_checker = SearchQualityChecker(queries_lemmas, pr_index)
pr_res = pr_quality_checker.get_results(pagerank_query)

In [ ]:
pr_res

In [ ]:
settings_all = {
    "mappings": {
        "properties": {
            "title": {
                "type": "text"
            },
            "content": {
                "type": "text"
            },
            "urllen": {
                "type": "rank_feature"
            },
            "doclen": {
                "type": "rank_feature"
            },
            "pagerank": {
                "type": "rank_feature"
            }
        }
    }
}


In [ ]:
all_index = Index("all_index", settings_all)

In [ ]:
all_index.add_documents("res/lemmatized_titles_pr_len")

In [ ]:
print_index_size(all_index)

In [ ]:
all_quality_checker = SearchQualityChecker(queries_lemmas, all_index)
all_res = all_quality_checker.get_results(all_query)

In [ ]:
all_res

In [ ]:
def all_query_creator(b1, b2, b3, b4):
    def all_query_in(query):
        return  {
        'query': {
            'bool': {
                'should': [
                    {
                        'match': {
                            'content': query.query
                        }
                    },
                    {
                        'match': {
                            'title': {
                                'query': query.query,
                                'boost': str(b1)
                            }
                        }
                    },
                    {
                        'rank_feature': {
                            'field': 'pagerank',
                            'saturation': {
                                'pivot': 10
                            },
                            'boost': str(b2)
                        }
                    },
                    {
                        'rank_feature': {
                            'field': 'urllen',
                            'boost': str(b3)
                    }
                    },
                    {
                        'rank_feature': {
                            'field': 'doclen',
                            'boost': str(b4)
                        }
                    }
                ]
            }
        }
    }
    
    return all_query_in

In [ ]:
all_ress = []
for b1 in [0.15, 0.3, 0.8, 1.5, 3, 5]:
    for b2 in [0.15, 0.3, 0.8, 1.5, 3, 5]:
        for b3 in [0.15, 0.3, 0.8, 1.5, 3, 5]:
            for b4 in [0.15, 0.3, 0.8, 1.5, 3, 5]:
                all_res = all_quality_checker.get_results(all_query_creator(b1, b2, b3, b4))
                all_ress.append(all_res)
                


In [ ]:
all_ress

In [ ]:
db1=0.15
db2=5.0
db3=0.5
db4=0.15

In [ ]:
all_ress1 = []
r =[]
p = []
rp = []
mapp = []
#b1l = [0.02, 0.05, 0.08, 0.11, 0.14, 0.17, 0.2, 0.23, 0.26, 0.29]
b1l = [0.2]
for b1 in b1l:
    all_res = all_quality_checker.get_results(all_query_creator(b1, db2, db3, db4))
    a, b, c, d = all_res.p, all_res.r, all_res.r_precision, all_res.map_score
    r.append(a)
    p.append(b)
    rp.append(c)
    mapp.append(d)
    all_ress.append(all_res)

In [ ]:
import matplotlib.pyplot as plt

plt.clf()
plt.xlabel('Boost')
plt.ylabel('r@20')
plt.plot(b1l, r)
plt.savefig("res/plots/r11.png")

plt.clf()
plt.xlabel('Boost')
plt.ylabel('p@20')
plt.plot(b1l, p)
plt.savefig("res/plots/p11.png")

plt.clf()
plt.xlabel('Boost')
plt.ylabel('r_prec')
plt.plot(b1l, rp)
plt.savefig("res/plots/rp11.png")

plt.clf()
plt.xlabel('Boost')
plt.ylabel('map')
plt.plot(b1l, mapp)
plt.savefig("res/plots/map11.png")

In [ ]:
all_ress1 = []
r =[]
p = []
rp = []
mapp = []
b1l = [0.01, 0.15, 0.8, 1.5, 5]
for b3 in b1l:
    all_res = all_quality_checker.get_results(all_query_creator(db1, db2, b3, db4))
    a, b, c, d = all_res.p, all_res.r, all_res.r_precision, all_res.map_score
    r.append(a)
    p.append(b)
    rp.append(c)
    mapp.append(d)
    all_ress.append(all_res)

In [ ]:
import matplotlib.pyplot as plt

plt.clf()
plt.xlabel('Boost')
plt.ylabel('r@20')
plt.plot(b1l, r)
plt.savefig("res/plots/r2.png")

plt.clf()
plt.xlabel('Boost')
plt.ylabel('p@20')
plt.plot(b1l, p)
plt.savefig("res/plots/p2.png")

plt.clf()
plt.xlabel('Boost')
plt.ylabel('r_prec')
plt.plot(b1l, rp)
plt.savefig("res/plots/rp2.png")

plt.clf()
plt.xlabel('Boost')
plt.ylabel('map')
plt.plot(b1l, mapp)
plt.savefig("res/plots/map2.png")

In [ ]:
all_ress1 = []
r =[]
p = []
rp = []
mapp = []
b1l = [0.01, 0.15, 0.8, 1.5, 5]
for b4 in b1l:
    all_res = all_quality_checker.get_results(all_query_creator(db1, db2, db3, b4))
    a, b, c, d = all_res.p, all_res.r, all_res.r_precision, all_res.map_score
    r.append(a)
    p.append(b)
    rp.append(c)
    mapp.append(d)
    all_ress.append(all_res)

In [ ]:
import matplotlib.pyplot as plt

plt.clf()
plt.xlabel('Boost')
plt.ylabel('r@20')
plt.plot(b1l, r)
plt.savefig("res/plots/r3.png")

plt.clf()
plt.xlabel('Boost')
plt.ylabel('p@20')
plt.plot(b1l, p)
plt.savefig("res/plots/p3.png")

plt.clf()
plt.xlabel('Boost')
plt.ylabel('r_prec')
plt.plot(b1l, rp)
plt.savefig("res/plots/rp3.png")

plt.clf()
plt.xlabel('Boost')
plt.ylabel('map')
plt.plot(b1l, mapp)
plt.savefig("res/plots/map3.png")